# Анализ качества вина: Машинное обучение для предсказания оценок экспертов

## Оглавление
1. [Введение](#введение)
2. [Загрузка и первичный анализ данных](#загрузка-и-первичный-анализ-данных)
3. [Исследовательский анализ данных (EDA)](#исследовательский-анализ-данных-eda)
4. [Подготовка данных](#подготовка-данных)
5. [Обучение моделей](#обучение-моделей)
6. [Оптимизация гиперпараметров](#оптимизация-гиперпараметров)
7. [Финальная оценка](#финальная-оценка)
8. [Выводы](#выводы)

In [8]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Машинное обучение
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.impute import SimpleImputer
import xgboost as xgb
import joblib

# Настройка отображения
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("Библиотеки успешно импортированы!")

Библиотеки успешно импортированы!


## 1. Введение

В данной работе мы анализируем датасет качества вина, содержащий химические показатели красных и белых вин, а также оценки качества экспертами по шкале от 3 до 9.

**Цель исследования:** Построить модель машинного обучения для предсказания качества вина на основе его химических характеристик.

**Источник данных:** [Wine Quality Dataset](https://www.kaggle.com/datasets/yasserh/wine-quality-dataset)

In [9]:
# Загрузка данных
# Предполагается, что файлы находятся в папке data/
red_wine = pd.read_csv('../data/winequality-red.csv')
white_wine = pd.read_csv('../data/winequality-white.csv')

# Добавляем тип вина
red_wine['wine_type'] = 'red'
white_wine['wine_type'] = 'white'

# Объединяем датасеты
wine_data = pd.concat([red_wine, white_wine], ignore_index=True)

print(f"Общий размер датасета: {wine_data.shape}")
print(f"Красное вино: {len(red_wine)} образцов")
print(f"Белое вино: {len(white_wine)} образцов")

FileNotFoundError: [Errno 2] No such file or directory: '../data/winequality-red.csv'

## 2. Загрузка и первичный анализ данных

In [10]:
# Первичный анализ структуры данных
print("=== ИНФОРМАЦИЯ О ДАТАСЕТЕ ===")
print(f"Размерность: {wine_data.shape}")
print(f"\nТипы данных:")
print(wine_data.dtypes)
print(f"\nПропущенные значения:")
print(wine_data.isnull().sum())
print(f"\nСтатистическое описание:")
wine_data.describe()

=== ИНФОРМАЦИЯ О ДАТАСЕТЕ ===


NameError: name 'wine_data' is not defined

In [ ]:
# Анализ целевой переменной
plt.figure(figsize=(15, 5))

# Распределение качества
plt.subplot(1, 3, 1)
wine_data['quality'].value_counts().sort_index().plot(kind='bar')
plt.title('Распределение оценок качества вина')
plt.xlabel('Качество')
plt.ylabel('Количество образцов')
plt.xticks(rotation=0)

# Распределение по типам вина
plt.subplot(1, 3, 2)
wine_data['wine_type'].value_counts().plot(kind='bar', color=['red', 'white'])
plt.title('Распределение по типам вина')
plt.xlabel('Тип вина')
plt.ylabel('Количество образцов')
plt.xticks(rotation=0)

# Качество по типам вина
plt.subplot(1, 3, 3)
quality_by_type = wine_data.groupby(['wine_type', 'quality']).size().unstack(fill_value=0)
quality_by_type.plot(kind='bar', stacked=True)
plt.title('Качество по типам вина')
plt.xlabel('Тип вина')
plt.ylabel('Количество образцов')
plt.xticks(rotation=0)
plt.legend(title='Качество', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

# Статистика по качеству
print("\n=== СТАТИСТИКА ПО КАЧЕСТВУ ===")
print(wine_data['quality'].value_counts().sort_index())
print(f"\nСреднее качество: {wine_data['quality'].mean():.2f}")
print(f"Медиана качества: {wine_data['quality'].median():.2f}")
print(f"Стандартное отклонение: {wine_data['quality'].std():.2f}")

## 3. Исследовательский анализ данных (EDA)

In [ ]:
# Корреляционный анализ
# Создаем числовую копию данных для корреляции
wine_numeric = wine_data.copy()
wine_numeric['wine_type_red'] = (wine_numeric['wine_type'] == 'red').astype(int)
wine_numeric = wine_numeric.drop('wine_type', axis=1)

# Матрица корреляции
plt.figure(figsize=(14, 10))
correlation_matrix = wine_numeric.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Матрица корреляции химических показателей вина')
plt.tight_layout()
plt.show()

# Корреляция с целевой переменной
quality_corr = correlation_matrix['quality'].abs().sort_values(ascending=False)
print("\n=== КОРРЕЛЯЦИЯ С КАЧЕСТВОМ (по модулю) ===")
for feature, corr in quality_corr.items():
    if feature != 'quality':
        print(f"{feature:<25}: {corr:.3f}")

In [ ]:
# Анализ распределений признаков
numeric_features = wine_numeric.columns.drop(['quality', 'wine_type_red'])

fig, axes = plt.subplots(4, 3, figsize=(18, 16))
axes = axes.ravel()

for idx, feature in enumerate(numeric_features):
    # Гистограмма
    axes[idx].hist(wine_numeric[feature], bins=30, alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'Распределение {feature}')
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Частота')
    
    # Добавляем статистики
    mean_val = wine_numeric[feature].mean()
    median_val = wine_numeric[feature].median()
    axes[idx].axvline(mean_val, color='red', linestyle='--', label=f'Среднее: {mean_val:.2f}')
    axes[idx].axvline(median_val, color='green', linestyle='--', label=f'Медиана: {median_val:.2f}')
    axes[idx].legend()

# Удаляем лишние подплоты
for idx in range(len(numeric_features), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

In [ ]:
# Анализ выбросов
fig, axes = plt.subplots(4, 3, figsize=(18, 16))
axes = axes.ravel()

outlier_stats = {}

for idx, feature in enumerate(numeric_features):
    # Boxplot
    axes[idx].boxplot(wine_numeric[feature])
    axes[idx].set_title(f'Выбросы в {feature}')
    axes[idx].set_ylabel(feature)
    
    # Вычисляем статистику выбросов по IQR
    Q1 = wine_numeric[feature].quantile(0.25)
    Q3 = wine_numeric[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = wine_numeric[(wine_numeric[feature] < lower_bound) | 
                          (wine_numeric[feature] > upper_bound)][feature]
    outlier_stats[feature] = {
        'count': len(outliers),
        'percentage': len(outliers) / len(wine_numeric) * 100,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }

# Удаляем лишние подплоты
for idx in range(len(numeric_features), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

# Выводим статистику выбросов
print("\n=== СТАТИСТИКА ВЫБРОСОВ (метод IQR) ===")
for feature, stats in outlier_stats.items():
    print(f"{feature:<25}: {stats['count']:>4} ({stats['percentage']:>5.1f}%)")

## 4. Подготовка данных

In [ ]:
# Подготовка признаков и целевой переменной
X = wine_numeric.drop('quality', axis=1)
y = wine_numeric['quality']

# Стратифицированное разделение данных
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp  # 0.176 ≈ 0.15/0.85
)

print("=== РАЗДЕЛЕНИЕ ДАННЫХ ===")
print(f"Обучающая выборка: {X_train.shape[0]} образцов ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Валидационная выборка: {X_val.shape[0]} образцов ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Тестовая выборка: {X_test.shape[0]} образцов ({X_test.shape[0]/len(X)*100:.1f}%)")

# Проверяем распределение классов
print("\nРаспределение классов:")
print("Обучающая:", y_train.value_counts().sort_index().values)
print("Валидационная:", y_val.value_counts().sort_index().values)
print("Тестовая:", y_test.value_counts().sort_index().values)

In [ ]:
# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Преобразуем обратно в DataFrame для удобства
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("=== МАСШТАБИРОВАНИЕ ПРИЗНАКОВ ===")
print("Применен StandardScaler")
print(f"Средние значения после масштабирования (должны быть ~0):")
print(X_train_scaled.mean().round(6))
print(f"\nСтандартные отклонения после масштабирования (должны быть ~1):")
print(X_train_scaled.std().round(6))

## 5. Обучение моделей

In [ ]:
# Baseline модели
models = {
    'DummyClassifier': DummyClassifier(strategy='most_frequent', random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'DecisionTree': DecisionTreeClassifier(random_state=42)
}

baseline_results = {}

print("=== ОБУЧЕНИЕ BASELINE МОДЕЛЕЙ ===")
for name, model in models.items():
    print(f"\nОбучение {name}...")
    
    # Обучение
    model.fit(X_train_scaled, y_train)
    
    # Предсказания
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    
    # Метрики
    train_accuracy = accuracy_score(y_train, y_pred_train)
    val_accuracy = accuracy_score(y_val, y_pred_val)
    
    baseline_results[name] = {
        'model': model,
        'train_accuracy': train_accuracy,
        'val_accuracy': val_accuracy
    }
    
    print(f"  Train Accuracy: {train_accuracy:.4f}")
    print(f"  Val Accuracy: {val_accuracy:.4f}")
    print(f"  Переобучение: {train_accuracy - val_accuracy:.4f}")

In [ ]:
# Продвинутые модели
advanced_models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42, eval_metric='mlogloss'),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

advanced_results = {}

print("\n=== ОБУЧЕНИЕ ПРОДВИНУТЫХ МОДЕЛЕЙ ===")
for name, model in advanced_models.items():
    print(f"\nОбучение {name}...")
    
    # Обучение с измерением времени
    import time
    start_time = time.time()
    model.fit(X_train_scaled, y_train)
    training_time = time.time() - start_time
    
    # Предсказания
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    
    # Метрики
    train_accuracy = accuracy_score(y_train, y_pred_train)
    val_accuracy = accuracy_score(y_val, y_pred_val)
    
    # Кросс-валидация
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    
    advanced_results[name] = {
        'model': model,
        'train_accuracy': train_accuracy,
        'val_accuracy': val_accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'training_time': training_time
    }
    
    print(f"  Train Accuracy: {train_accuracy:.4f}")
    print(f"  Val Accuracy: {val_accuracy:.4f}")
    print(f"  CV Mean ± Std: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    print(f"  Training Time: {training_time:.2f}s")
    print(f"  Переобучение: {train_accuracy - val_accuracy:.4f}")

## 6. Оптимизация гиперпараметров

In [ ]:
# Выбираем лучшую модель для тюнинга
best_model_name = max(advanced_results.keys(), 
                     key=lambda x: advanced_results[x]['val_accuracy'])
print(f"Лучшая модель для тюнинга: {best_model_name}")
print(f"Валидационная точность: {advanced_results[best_model_name]['val_accuracy']:.4f}")

# Определяем параметры для тюнинга в зависимости от модели
if best_model_name == 'XGBoost':
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5, 6],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    }
    base_model = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss')
    
elif best_model_name == 'RandomForest':
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    base_model = RandomForestClassifier(random_state=42, n_jobs=-1)
    
else:
    param_grid = {}
    base_model = advanced_models[best_model_name]

# Выполняем тюнинг
if param_grid:
    print(f"\nВыполняем тюнинг гиперпараметров для {best_model_name}...")
    
    # Используем RandomizedSearchCV для экономии времени
    from sklearn.model_selection import RandomizedSearchCV
    
    random_search = RandomizedSearchCV(
        base_model, param_grid, n_iter=50, cv=3, 
        scoring='accuracy', random_state=42, n_jobs=-1, verbose=1
    )
    
    random_search.fit(X_train_scaled, y_train)
    
    # Лучшая модель
    best_model = random_search.best_estimator_
    print(f"\nЛучшие параметры: {random_search.best_params_}")
    print(f"Лучший CV score: {random_search.best_score_:.4f}")
    
else:
    best_model = advanced_models[best_model_name]
    print(f"Используем модель {best_model_name} без дополнительного тюнинга")

## 7. Финальная оценка

In [ ]:
# Финальная оценка на тестовой выборке
y_test_pred = best_model.predict(X_test_scaled)
y_test_proba = best_model.predict_proba(X_test_scaled)

# Метрики
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score

test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"=== ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ ===")
print(f"Модель: {best_model_name}")
print(f"Точность на тестовой выборке: {test_accuracy:.4f}")

# Подробный отчет по классификации
print(f"\nОтчет по классификации:")
print(classification_report(y_test, y_test_pred))

# Матрица ошибок
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=sorted(y.unique()),
            yticklabels=sorted(y.unique()))
plt.title('Матрица ошибок')
plt.xlabel('Предсказанное качество')
plt.ylabel('Реальное качество')
plt.show()

# ROC AUC для многоклассовой классификации
roc_auc = roc_auc_score(y_test, y_test_proba, multi_class='ovr')
print(f"\nROC AUC (One-vs-Rest): {roc_auc:.4f}")

In [ ]:
# Важность признаков
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feature_importance, x='importance', y='feature')
    plt.title('Важность признаков')
    plt.xlabel('Важность')
    plt.tight_layout()
    plt.show()
    
    print("\n=== ВАЖНОСТЬ ПРИЗНАКОВ ===")
    for idx, row in feature_importance.iterrows():
        print(f"{row['feature']:<25}: {row['importance']:.4f}")

# Сохранение модели
import os
os.makedirs('../models', exist_ok=True)

joblib.dump(best_model, '../models/best_wine_model.pkl')
joblib.dump(scaler, '../models/scaler.pkl')

# Сохранение метаданных модели
import json
metadata = {
    'model_type': best_model_name,
    'test_accuracy': float(test_accuracy),
    'roc_auc': float(roc_auc),
    'feature_names': list(X_train.columns),
    'classes': list(sorted(y.unique())),
    'training_size': len(X_train),
    'features_count': len(X_train.columns)
}

with open('../models/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\nМодель сохранена в ../models/best_wine_model.pkl")
print(f"Скейлер сохранен в ../models/scaler.pkl")
print(f"Метаданные сохранены в ../models/model_metadata.json")

## 8. Выводы

### Основные результаты:

1. **Качество данных**: Датасет содержит 6497 образцов вина без пропущенных значений
2. **Дисбаланс классов**: Большинство вин имеют качество 5-6 баллов
3. **Лучшая модель**: Достигнута точность {test_accuracy:.1%} на тестовой выборке
4. **Важные признаки**: Содержание алкоголя и летучая кислотность наиболее влияют на качество

### Рекомендации:

1. Собрать больше данных для редких классов качества (3-4, 8-9)
2. Рассмотреть использование методов работы с несбалансированными данными
3. Добавить дополнительные признаки (регион, год урожая, сорт винограда)
4. Исследовать ансамблевые методы для улучшения качества предсказаний

### Практическое применение:

Модель может быть использована виноделами для:
- Контроля качества в процессе производства
- Оптимизации химического состава вина
- Предварительной оценки качества перед дегустацией экспертами